Устанавливаем scholarly.



In [ ]:
pip install scholarly

In [3]:
from scholarly import scholarly
from scholarly import ProxyGenerator
import os
import csv

Так как Google Scholar позволяет выкачивать ограниченное количество данных за сутки, нам будет необходимо с каждым новым запросом обновлять прокси.

In [4]:
pg = ProxyGenerator()
scholarly.use_proxy(pg, pg)

Мы будем производить поиск по ID каждго автора, чтобы избежать проблемы с полными тёзками. ID есть на странице автора в Google Scholar. В список author_ids записываем интересующие нас ID.

In [5]:
author_ids = ['D1HB8BAAAAAJ']

Заводим словарь, куда будем сохранять нужную нам информацию - имя автора, название публикации, год публикации, количество соавторов, имена соавторов.
Заводим файл в формате csv, где в качестве заголовков столбцов будут выступать ключи из словаря.

Для каждого автора из списка author_ids с помощью функции scholarly.fill() ищем публикации. В переменные name и publications записываем имя и публикации.
Сравниваем количество уже имеющихся публикаций с количеством публикаций в google scholar, если имеющихся больше или столько же, переходим к следующему автору.
В цикле смотрим каждую публикацию. Если публикация уже есть в csv файле, пропускаем её. С помощью scholarly.fill() получаем данные о публикации. В словарь pub_res сохраняем данные об имени автора, названии публикации, количестве соавторов, именах соавторов и годе публикации.
Информацию из словаря записываем в csv файл.

In [7]:
keys = ['author','title','pub_year','number_of_co_authors','co_authors']

output_file =  open('publications.csv', 'a', newline='')
dict_writer = csv.DictWriter(output_file, keys)
if os.path.getsize('publications.csv') == 0:
    dict_writer.writeheader()
    output_file.flush()
dict_reader = csv.DictReader(open('publications.csv', newline=''), keys)
csv_reader = [row for row in dict_reader]



for id in author_ids:
    author = scholarly.search_author_id('D1HB8BAAAAAJ')
    author =  scholarly.fill((author),sections = ['publications'])
    name = author['name']
    pubs = author['publications']
    author_pubs_in_file = [row for row in csv_reader if row['author'] == name]
    num_author_pubs_in_file = len(author_pubs_in_file)
    if num_author_pubs_in_file >= len(author['publications']): continue
    i = 0
    for pub in pubs:
        if i < num_author_pubs_in_file:
            i += 1
            continue
        pub = scholarly.fill(pub)
        if len([x for x in author_pubs_in_file if x['title'] == pub['bib']['title']]) > 0 : continue
        pub_res = {
            "author":name,
            "title":pub['bib']['title']
            }

        if 'author' in pub['bib']:
            pub_res['number_of_co_authors'] = len(pub['bib']['author'].split(' and ')) - 1
            pub_res['co_authors'] = pub['bib']['author']
        else:
            pub_res["number_of_co_authors"] = ''

        if 'pub_year' in pub['bib']:
            pub_res['pub_year'] = pub['bib']['pub_year']
        else:
            pub_res['pub_year'] = ''

        dict_writer.writerows([pub_res])
        output_file.flush()